# Horse Racing Classification
### Jamie Heneghan, Melka Konshie, Kyle Duffy, Walter Robson, Sam Allison
### CSE 40647

**Getting the Data**

In [ ]:
import pandas as pd
import numpy as np
runs = pd.read_csv("./runs.csv")
races = pd.read_csv("./races.csv")

print(runs.shape)
print(races.shape)

In [ ]:
runs

In [ ]:
races.head()

In [ ]:
## Merge based on horse id, historical

In [ ]:
historical_df_runs = runs[['won', 'horse_type','finish_time','horse_id','race_id']]

historical_df_races = races[['race_id', 'venue', 'config', 'distance', 'going', 'race_class']]

In [ ]:
historical_df_runs.join(historical_df_runs.groupby('horse_id')['finish_time'].agg(['mean']), on='horse_id')

In [ ]:
historical_df = pd.merge(historical_df_runs, historical_df_races, on='race_id')

In [ ]:
#historical_df.head()

#historical_df = historical_df.groupby('horse_id')
## For to use this, you can do something like this:
#avg_times = historical_df.groupby('horse_id')['finish_time'].mean() -> gets the average of finish time for horses with 
avg_times = historical_df.groupby('horse_id')['finish_time'].mean()
avg_times.head()

In [ ]:
## this kinda returns a df of dfs if that makes sense. Still gotta make it last 3, not all
historical_df = historical_df.groupby('horse_id').agg(lambda x: x.tolist())

In [ ]:
historical_df.head()

In [ ]:
## Take the features we want (This can be changed)

In [ ]:
runs_features = runs[['race_id', 'won', 'horse_age', 'horse_country', 'horse_type',
                      'horse_rating', 'horse_gear', 'declared_weight', 'actual_weight',
                      'draw', 'win_odds', 'horse_id', 'finish_time']]

races_features = races[['race_id', 'venue','date', 'config', 'distance', 'going', 'race_class']]


In [ ]:
## Merge the races and runs by race ID

In [ ]:
df = pd.merge(runs_features, races_features, on='race_id')

In [ ]:
## Add melkas stuff here

In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
df['speed'] = df['distance'] / df['finish_time']

In [ ]:
historical_averages = pysqldf('''
        SELECT a.horse_id, a.race_id, avg(b.finish_time) as average_past_finish_time, avg(b.speed) as average_past_speed, count(*)+1 as race_count, sum(b.won) as past_wins
        FROM df a, df b 
        where a.horse_id = b.horse_id
        and a.race_id != b.race_id
        and a.date > b.date
        group by a.horse_id, a.race_id
        '''
        )

In [ ]:
historical_averages.shape
df = df.merge(historical_averages, on=['horse_id', 'race_id'], how='left')

In [ ]:
df["average_past_finish_time"] = df.groupby("horse_id")["average_past_finish_time"].transform(lambda x: x.fillna(x.mean()))
df["average_past_speed"]    = df.groupby("horse_id")["average_past_speed"].transform(lambda x: x.fillna(x.mean()))
df["race_count"]            = df.groupby("horse_id")["race_count"].transform(lambda x: x.fillna(1))
df["past_wins"]             = df.groupby("horse_id")["past_wins"].transform(lambda x: x.fillna(0))
df["past_win_percentege"]   = df["past_wins"] / df["race_count"]

In [ ]:
df = df.drop('date', axis=1)
df = df.dropna()

In [ ]:
df = df.join(df.groupby('horse_id')['finish_time'].agg(['mean']), on='horse_id')

In [ ]:
## Drop Missing Values (this can be changed too, just easier this way)

In [ ]:
df = df.dropna()

In [ ]:
#df = df.join(df.groupby('horse_id')['finish_time'].agg(['mean']), on='horse_id')
## i deleted this above line idk why
#df = df.drop(columns=['finish_time'])
#df.columns

In [ ]:
df = df.drop(columns=['finish_time'])


In [ ]:
df.shape
df.head()

**Clean & Process the Data**

In [ ]:
## Convert gear -> binary
df['horse_gear'] = np.where(df['horse_gear'] == '--', 0, 1)

df.columns

In [ ]:
## Categorical -> Numerical (one hot encoding)
df = pd.get_dummies(df)

In [ ]:
df.head()

In [ ]:
df.shape
df.columns

## Try and Group by Race, and then predict each race not individual horses

In [ ]:
df_byRace = df.groupby('race_id').agg(lambda x: x.tolist())
df_byRace.head()
##this is good (i think)


In [ ]:
df_byRace.shape

In [ ]:
## gonna use first 5000 as train, last 1348 as test (races)

### Models

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## This is predicting whether a horse wins the race or not.
X_data = df.drop(columns=['won','win_odds'])
#drop 'won' cause that's the Y, drop race_id cause its noise?
## Theres definitely a better way to do this. We should be predicting which horse out of n horses in race x wins, not if horse is a winner in general
## Maybe we can compare the "likelihood of winning" for all n horses in race X, choose winner based on that, but then we need a model which gives us likelihood

## I hope this works, im trying to get a df of only the win_odds for testing purposes later
Y_data = df[['won', 'win_odds', 'race_id', 'horse_id']]

#df1 = df[df['Sales'] >= s]
X_train = X_data[X_data['race_id'] < 5000]
X_test = X_data[X_data['race_id'] >= 5000]

y_train = Y_data[Y_data['race_id'] < 5000]
y_test = Y_data[Y_data['race_id'] >= 5000]

#Y_data.head
#Y_train.head
## Probably a better sampling method, but idk this is easiest
#X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=.2, random_state=69)

#print(y_test)
only_odds = y_test#.drop(columns=['won'])
y_train = y_train.drop(columns=['win_odds', 'race_id'])
#y_train = y_train.values.ravel()
#print(y_train)
y_test = y_test.drop(columns=['win_odds', 'race_id', 'horse_id'])
y_test = y_test.values.ravel()
#only_odds = only_odds.values.ravel()


#print(only_odds)




In [ ]:
X_test_byr = X_test.groupby('race_id').agg(lambda x: x.tolist())
only_odds_byr = only_odds.groupby('race_id').agg(lambda x: x.tolist())
X_test_byr.head()
only_odds_byr.head()

In [ ]:
#y_test = y_test.values.ravel()
print(y_train)


In [ ]:
print("X_train.shape", X_train.shape)
print("y_train.shape", len(y_train['won']), "#Winners:", sum(y_train['won']))
print("X_test.shape", X_test.shape)
print("y_test.shape", len(y_test), "#Winners:", sum(y_test))


In [ ]:
X_train = X_train.drop(columns=['race_id'])
X_test = X_test.drop(columns=['race_id'])
y_train = y_train.drop(columns=['horse_id'])
#y_test = y_test.drop(columns=['horse_id'])
y_train = y_train.values.ravel()

In [ ]:
print("X_train.shape", X_train.shape)
print("y_train.shape", y_train.shape, "#Winners:", sum(y_train))
print("X_test.shape", X_test.shape)
print("y_test.shape", len(y_test), "#Winners:", sum(y_test))
#y_train.head()


In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=8)
clf_id3 = tree.DecisionTreeClassifier(criterion="entropy", max_depth=10)
clf_id3_overfit = tree.DecisionTreeClassifier(criterion="entropy")
clf_cart = tree.DecisionTreeClassifier(max_depth=10)
clf_bayes = GaussianNB()
clf_rbf = SVC(probability=True)
clf_forest = RandomForestClassifier()
clf_boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=2), random_state=69)
clf_mlp = MLPClassifier(hidden_layer_sizes=(50,50,), max_iter=1000, tol=0.001, random_state=420)

clf_knn.fit(X_train, y_train)
clf_id3.fit(X_train, y_train)
clf_id3_overfit.fit(X_train, y_train)
clf_cart.fit(X_train, y_train)
clf_bayes.fit(X_train, y_train)
clf_rbf.fit(X_train, y_train)
clf_forest.fit(X_train, y_train)
clf_boost.fit(X_train, y_train)
clf_mlp.fit(X_train, y_train)

y_pred_knn = clf_knn.predict(X_test)
y_pred_id3 = clf_id3.predict(X_test)
y_pred_id3_of = clf_id3_overfit.predict(X_test)
y_pred_cart = clf_cart.predict(X_test)
y_pred_bayes = clf_bayes.predict(X_test)
y_pred_rbf = clf_rbf.predict(X_test)
y_pred_forest = clf_forest.predict(X_test)
y_pred_boost = clf_boost.predict(X_test)
y_pred_mlp = clf_mlp.predict(X_test)


In [ ]:
weights = {1:.1, 0:.9}

clf_ridge = linear_model.RidgeClassifier(alpha=1, class_weight='balanced')
clf_ridge.fit(X_train, y_train)
y_pred_ridge = clf_ridge.predict(X_test)
print(X_train.columns)

In [ ]:
print("- Ridge:\n", metrics.classification_report(y_test, y_pred_ridge))

In [ ]:
only_odds = only_odds.values.ravel()

In [ ]:
bankroll_ridge = 0
### idk what this does tbh
#print(y_test.shape)
#print(only_odds.shape)
#print(y_test[61253])
i = 0
#only_odds = only_odds.values.ravel()
predicted_yes = 0
predicted_no = 0
correct_winners = 0
for index, run in X_test.iterrows():
    #print(index, run)
    #print(index)
    #bet a dollar to place (need to change this to win...)
    prediction_knn = clf_ridge.predict([run])
    #print("prediction", prediction_knn,"result", y_test[i], "odds", only_odds[i], "bankroll", bankroll_ridge)
    # Todo: We can graph the bankroll overtime for visuals
    
    #print(prediction)
    #print(only_odds[i])
    odds = only_odds[1 + i*4] #this is messy but works
    #print(prediction_knn, y_test[i], odds, ba)
    if prediction_knn:
        predicted_yes += 1
        if (y_test[i] == 1):
            correct_winners += 1
            bankroll_ridge += odds
        else:
            bankroll_ridge -= 1
    else:
        predicted_no += 1
            
    i += 1
    
print("Predicted Winners", predicted_yes, "Predicted Losers", predicted_no, "Correct Winners", correct_winners)

In [ ]:
print(bankroll_ridge)
print(len(y_test))

# Try to test on Races here, hopefully

In [ ]:
test = 1
#print(X_test_byr.columns)
#print(only_odds_byr.index.name)

predicted_loss = 0
predicted_win = 0
correct_p_win = 0
incorrect_p_win = 0

bankroll_ridge_byr = 0

max_winnings = 0

for feature, run in X_test_byr.iterrows():
    df = pd.DataFrame(run.values.tolist(), index=run.index)
    df = df.transpose()
    #print(feature)
    ##good 
    predictions = []
    odds_arr = []
    winners = []
    for index, run in df.iterrows():
        #print(run)
        d = clf_ridge.decision_function([run])
        prediction = np.interp(d[0],[-1,1],[0,1])
        predictions.append(prediction)
        ## this is working, now we just have to implement the odds
        odds = only_odds_byr.loc[feature]
        odds_arr.append(odds['win_odds'][index])
        winners.append(odds['won'][index])
        
        d = clf_ridge.decision_function([run])
        prediction_ridge = np.interp(d[0],[-1,1],[0,1])
    
    
    ##above gets probs of each horse in the race, below attempts to bet based on that
    total_odds = sum(predictions)
    for i in range(len(predictions)):
        predictions[i] = predictions[i]/total_odds
        implied_odds = 1/odds_arr[i]
        if predictions[i] > implied_odds * 1.1: # need 10% adv to bet?
            predicted_win += 1
            if winners[i] == 1:
                bankroll_ridge_byr += odds_arr[i]
                if odds_arr[i] > max_winnings:
                    max_winnings = odds_arr[i]
                    print(feature,"\n",run,"\nODDS---------",odds_arr[i], "\nprediction------",predictions[i])
                correct_p_win += 1
            else:
                bankroll_ridge_byr -= 1
                incorrect_p_win += 1
        else:
            predicted_loss += 1
        #print("Implied Odds", round(implied_odds, 3), "\tpredicted proba", round(predictions[i], 3), "\tBankroll $", bankroll_ridge_byr)
        
        
##its working
#print(run['horse_age'])
#df.columns
#only_odds_byr.head()

In [ ]:
print("$",bankroll_ridge_byr)
print("Correct Wins", correct_p_win, "Incorrect Wins", incorrect_p_win)

In [ ]:
## try with another classifier maybe, this is just copy pasted from above
test = 1
#print(X_test_byr.columns)
#print(only_odds_byr.index.name)

predicted_loss = 0
predicted_win = 0
correct_p_win = 0
incorrect_p_win = 0

bankroll_ridge_byr = 0

for feature, run in X_test_byr.iterrows():
    df = pd.DataFrame(run.values.tolist(), index=run.index)
    df = df.transpose()
    #print(feature)
    ##good 
    predictions = []
    odds_arr = []
    winners = []
    for index, run in df.iterrows():
        #print(run)
        ####
        #### Replace clf_{forest}.predict_proba... with any of our other models to test them.
        ####
        d = clf_mlp.predict_proba([run])[0][1]
        #prediction = np.interp(d[0],[-1,1],[0,1])
        #print(d[0][1])
        predictions.append(d)
        ##sick this is working, now we just have to implement the odds
        odds = only_odds_byr.loc[feature]
        odds_arr.append(odds['win_odds'][index])
        winners.append(odds['won'][index])
        
        #d = clf_ridge.decision_function([run])
        #prediction_ridge = np.interp(d[0],[-1,1],[0,1])
    
    
    ##above gets probs of each horse in the race, below attempts to bet based on that
    total_odds = sum(predictions)
    for i in range(len(predictions)):
        #if total_odds == 0:
            #print(total_odds)
        predictions[i] = predictions[i]/total_odds
        implied_odds = 1/odds_arr[i]
        #print(predictions)
        if predictions[i] > implied_odds * 1.1: # need 10% adv to bet?
            predicted_win += 1
            if winners[i] == 1:
                bankroll_ridge_byr += odds_arr[i]
                correct_p_win += 1
            else:
                bankroll_ridge_byr -= 1
                incorrect_p_win += 1
        else:
            predicted_loss += 1
        #print("Implied Odds", round(implied_odds, 3), "\tpredicted proba", round(predictions[i], 3), "\tBankroll $", bankroll_ridge_byr)
        
        
##its working
#print(run['horse_age'])
#df.columns
#only_odds_byr.head()

In [ ]:
print("$",bankroll_ridge_byr)
print("Correct Wins", correct_p_win, "Incorrect Wins", incorrect_p_win)

In [ ]:
print("Accuracy:")
print("- KNN:", metrics.accuracy_score(y_test, y_pred_knn))
print("- ID3:", metrics.accuracy_score(y_test, y_pred_id3))
print("- ID3 (overfitting):", metrics.accuracy_score(y_test, y_pred_id3_of))
print("- CART:", metrics.accuracy_score(y_test, y_pred_cart))
print("- Naive Bayes:", metrics.accuracy_score(y_test, y_pred_bayes))
print("- RBF Kernel SVC:", metrics.accuracy_score(y_test, y_pred_rbf))
print("- Random Forest:", metrics.accuracy_score(y_test, y_pred_forest))
print("- AdaBoost:", metrics.accuracy_score(y_test, y_pred_boost))
print("- MLP:", metrics.accuracy_score(y_test, y_pred_mlp))
print("- Ridge:", metrics.accuracy_score(y_test, y_pred_ridge))

In [ ]:
## More Metrics!
print("- KNN:\n", metrics.classification_report(y_test, y_pred_knn))
print("- ID3:\n", metrics.classification_report(y_test, y_pred_id3))
print("")
print("- ID3 (overfitting):\n", metrics.classification_report(y_test, y_pred_id3_of))
print("")
print("- CART:\n", metrics.classification_report(y_test, y_pred_cart))
print("")
print("- Naive Bayes:\n", metrics.classification_report(y_test, y_pred_bayes))
print("")
print("- RBF Kernel SVC:\n", metrics.classification_report(y_test, y_pred_rbf))
print("")
print("- Random Forest:\n", metrics.classification_report(y_test, y_pred_forest, zero_division=0))
print("")
print("- AdaBoost:\n", metrics.classification_report(y_test, y_pred_boost))
print("")
print("- MLP:\n", metrics.classification_report(y_test, y_pred_mlp))
print("")

In [ ]:
## Maybe we should do regression on 'won' feature so we can get a probability value instead of binary value
## Idk, this is just a start

In [ ]:
#for i in range(len(only_odds)):
    #print(only_odds[i])

### Betting \$1 on Every Predicted Winner

In [ ]:
## loop through every run in the test data
## predict whether its a winner or not, and then use the odds of the run to add/subtract profit
bankroll_knn = 0
bankroll_id3 = 0
bankroll_id3_of = 0
bankroll_cart = 0
bankroll_bayes = 0
bankroll_rbf = 0
bankroll_forest = 0
bankroll_boost = 0
bankroll_mlp = 0

#print(y_test.shape)
#print(only_odds.shape)
#print(y_test[61253])
i = 0

for index, run in X_test.iterrows():
    #print(index)
    #bet a dollar to place (need to change this to win...)
    prediction_knn = clf_knn.predict([run])
    prediction_id3 = clf_id3.predict([run])
    prediction_id3_of = clf_id3_overfit.predict([run])
    prediction_cart = clf_cart.predict([run])
    prediction_bayes = clf_bayes.predict([run])
    prediction_rbf = clf_rbf.predict([run])
    prediction_forest = clf_forest.predict([run])
    prediction_boost = clf_boost.predict([run])
    prediction_mlp = clf_mlp.predict([run])
    
    #print(y_test[i])
    
    #print(prediction)
    odds = only_odds[1 +i*4]
    #print(odds)
    if prediction_knn:
        if (y_test[i] == 1):
            bankroll_knn += odds
        else:
            bankroll_knn -= 1
            
    if prediction_id3:
        if (y_test[i] == 1):
            bankroll_id3 += odds
        else:
            bankroll_id3 -= 1
    
    if prediction_id3_of:
        if (y_test[i] == 1):
            bankroll_id3_of += odds
        else:
            bankroll_id3_of -= 1
    
    if prediction_cart:
        if (y_test[i] == 1):
            bankroll_cart += odds
        else:
            bankroll_cart -= 1
    
    if prediction_bayes:
        if (y_test[i] == 1):
            bankroll_bayes += odds
        else:
            bankroll_bayes -= 1
            
    if prediction_rbf:
        if (y_test[i] == 1):
            bankroll_rbf += odds
        else:
            bankroll_rbf -= 1
            
    if prediction_forest:
        if (y_test[i] == 1):
            bankroll_forest += odds
        else:
            bankroll_forest -= 1
            
    if prediction_boost:
        if (y_test[i] == 1):
            bankroll_boost += odds
        else:
            bankroll_boost -= 1
    
    if prediction_mlp:
        if (y_test[i] == 1):
            bankroll_mlp += odds
        else:
            bankroll_mlp -= 1

    i += 1
    

        

In [ ]:
print("-----Betting 1 on Every Predicted Winner-----")
print("Bankroll KNN: $", round(bankroll_knn,2))
print("Bankroll ID3: $", round(bankroll_id3,2))
print("Bankroll ID3_OF: $", round(bankroll_id3_of,2))
print("Bankroll CART: $", round(bankroll_cart,2))
print("Bankroll Bayes: $", round(bankroll_bayes,2))
print("Bankroll rbf: $", round(bankroll_rbf,2))
print("Bankroll Forest: $", round(bankroll_forest,2))
print("Bankroll boost: $", round(bankroll_boost,2))
print("Bankroll mlp: $", round(bankroll_mlp,2))

## Betting $1 on Every Horse whose "True Odds" are better than Odds Given

In [ ]:
## loop through every run in the test data
## predict whether its a winner or not, and then use the odds of the run to add/subtract profit

# I only really care about TP and FP cause we only bet on a predicted positive
bankroll_knn = 0
knn_TP, knn_FP = 0, 0

bankroll_id3 = 0
id3_TP, id3_FP = 0, 0

bankroll_id3_of = 0
id3_of_TP, id3_of_FP = 0, 0

bankroll_cart = 0
cart_TP, cart_FP = 0, 0

bankroll_bayes = 0
bayes_TP, bayes_FP = 0, 0

bankroll_rbf = 0
rbf_TP, rbf_FP = 0, 0

bankroll_forest = 0
forest_TP, forest_FP = 0, 0

bankroll_boost = 0
boost_TP, boost_FP = 0, 0

bankroll_mlp = 0
mlp_TP, mlp_FP = 0, 0

#print(y_test.shape)
#print(only_odds.shape)
#print(y_test[61253])
i = 0

print(only_odds[0])

#Probably a better way to do this
for index, run in X_test.iterrows():
    #print(index)
    #bet a dollar to place (need to change this to win...)
    prediction_knn = clf_knn.predict_proba([run])
    prediction_id3 = clf_id3.predict_proba([run])
    prediction_id3_of = clf_id3_overfit.predict_proba([run])
    prediction_cart = clf_cart.predict_proba([run])
    prediction_bayes = clf_bayes.predict_proba([run])
    prediction_rbf = clf_rbf.predict_proba([run])
    prediction_forest = clf_forest.predict_proba([run])
    prediction_boost = clf_boost.predict_proba([run])
    prediction_mlp = clf_mlp.predict_proba([run])
    
    #print(prediction)
    
    ## Decimal odds -> implied odds = 1/(decimal odds)
    implied_odds = 1.0/only_odds[1 +i*4]
    odds = only_odds[1 +i*4]
    
    
    if prediction_knn[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_knn += odds
            knn_TP += 1
        else:
            bankroll_knn -= 1
            knn_FP += 1
            
    if prediction_id3[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_id3 += odds
            id3_TP += 1
        else:
            bankroll_id3 -= 1
            id3_FP += 1
    
    if prediction_id3_of[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_id3_of += odds
            id3_of_TP += 1
        else:
            bankroll_id3_of -= 1
            id3_of_FP += 1
    
    if prediction_cart[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_cart += odds
            cart_TP += 1
        else:
            bankroll_cart -= 1
            cart_FP += 1
    
    if prediction_bayes[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_bayes += odds
            bayes_TP += 1
        else:
            bankroll_bayes -= 1
            bayes_FP += 1
            
    if prediction_rbf[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_rbf += odds
            rbf_TP += 1
        else:
            bankroll_rbf -= 1
            rbf_FP += 1
            
    if prediction_forest[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_forest += odds
            forest_TP += 1
        else:
            bankroll_forest -= 1
            forest_FP += 1
            
    if prediction_boost[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_boost += odds
            boost_TP += 1
        else:
            bankroll_boost -= 1
            boost_FP += 1
    
    if prediction_mlp[0][1] > implied_odds:
        if (y_test[i] == 1):
            bankroll_mlp += odds
            mlp_TP += 1
        else:
            bankroll_mlp -= 1
            mlp_FP += 1

    i += 1
    


In [ ]:
print("-----Betting 1 on Every Predicted Winner-----")
print("Bankroll KNN: $", round(bankroll_knn,2))
print("Bankroll ID3: $", round(bankroll_id3,2))
print("Bankroll ID3_OF: $", round(bankroll_id3_of,2))
print("Bankroll CART: $", round(bankroll_cart,2))
print("Bankroll Bayes: $", round(bankroll_bayes,2))
print("Bankroll rbf: $", round(bankroll_rbf,2))
print("Bankroll Forest: $", round(bankroll_forest,2))
print("Bankroll boost: $", round(bankroll_boost,2))
print("Bankroll mlp: $", round(bankroll_mlp,2),"\n")

print("KNN TP: ", knn_TP, "KNN FP: ", knn_FP)
print("ID3 TP: ", id3_TP, "ID3 FP: ", id3_FP)
print("ID3_OF TP: ", id3_of_TP, "ID3_OF FP: ", id3_of_FP)
print("CART TP: ", cart_TP, "CART FP: ", cart_FP)
print("BAYES TP: ", bayes_TP, "BAYES FP: ", bayes_FP)
print("rbf TP: ", rbf_TP, "rbf FP: ", rbf_FP)
print("forest TP: ", forest_TP, "forest FP: ", forest_FP)
print("boost TP: ", boost_TP, "boost FP: ", boost_FP)
print("mlp TP: ", mlp_TP, "mlp FP: ", mlp_FP)

In [ ]:
# so that didn't work ^
# We can try using a betting Criterion (Kelly Criterion) which changes amount bet based on perceived advantage.
#
# we could also try regression on time to complete the race, and compare the predicted time for each horse in a race
#
# we should also optimize the models, experiment with more / different features (avg speed of last N races for example)
#
# Features used in Bolton and Chapman:
# Lifetime Win %, Average Speed Rating, Winnings/Race past year, Weight, Post Position, New Distance?, Last Speed Rating, Jockey Win Percent, Jockey Win #
# I think most of these can be calculated form our data. 

#this is just a start

In [ ]:
##linear regression time
#simple betting strategy for ridge (linear regression)

In [ ]:
bankroll_ridge = 0
ridge_TP, ridge_FP = 0, 0

#print(y_test.shape)
#print(only_odds.shape)
#print(y_test[61253])
i = 0

print(only_odds[0])

#Probably a better way to do this
for index, run in X_test.iterrows():
    #print(index)
    #bet a dollar to place (need to change this to win...)
    d = clf_ridge.decision_function([run])
    prediction_ridge = np.interp(d[0],[-1,1],[0,1])
    
    
    #print(prediction)
    
    ## Decimal odds -> implied odds = 1/(decimal odds)
    implied_odds = 1.0/only_odds[1 +i*4]
    odds = only_odds[1 + i*4]
    
    
    if prediction_ridge > implied_odds:
        if (y_test[i] == 1):
            bankroll_ridge += odds
            ridge_TP += 1
        else:
            bankroll_ridge -= 1
            ridge_FP += 1
            
            
    

    i += 1
    

In [ ]:
print("Ridge (linear regression)$", bankroll_ridge)

In [ ]:
print("Ridge TP:", ridge_TP)
print("Ridge FP:", ridge_FP)

In [ ]:
## I think ridge (simple linear regression) is still best, it gives us an actual probability.
#  Our main focus should be finding a way to compare horses in the same race, as thats what we'd actually do at the track
#  Once that's done, we can figure out betting strategies etc and evaluation metrics. Doesn't necessarily have to be profitable, 
#  we just have to be able to show that we put some work in on the project. 
#  
#  If anyone wants to work on this, we need to reorganize the data in a way where we can organize by race_id. 
#  look into df.groupby, we can group by raceID and then loop through each race ID and predict for each horse
#  We should also seperate train and test by race, because now the training and testing can both feature runs from the same race.
#  We can also look into balancing the labels in training data, and sorting train/test chronologically, but not high priority I think.

